# 신용카드 사용자 연체 예측 AI 경진대회

### 데이터 설명
- `train.csv`: 신용카드 사용자들의 개인 신상정보 (26457, 20)   
- `test.csv`: credit 열 미포함 (10000, 19)   
- `sample_submission.csv`: 정답 제출 파일 (10000, 4)

### 데이터 변수 설명
- `index`
- `gender`: 성별
- `car`: 차량 소유 여부
- `reality`: 부동산 소유 여부
- `child_num`: 자녀 수
- `income_total`: 연간 소득
- `income_type`: 소득 분류 ['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
- `edu_type`: 교육 수준 ['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']
- `family_type`: 결혼 여부 ['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']
- `house_type`: 생활 방식 ['Municipal apartment', 'House / apartment', 'With parents', 'Co-op apartment', 'Rented apartment', 'Office apartment']
- `DAYS_BIRTH`: 출생일 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미)
- `DAYS_EMPLOYED`: 업무 시작일 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미, 양수 값은 고용되지 않은 상태를 의미함)
- `FLAG_MOBIL`: 핸드폰 소유 여부
- `work_phone`: 업무용 전화 소유 여부
- `phone`: 전화 소유 여부
- `email`: 이메일 소유 여부
- `occyp_type`: 직업 유형													
- `family_size`: 가족 규모
- `begin_month`: 신용카드 발급 월 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미)
- `credit`: 사용자의 신용카드 대금 연체를 기준으로 한 신용도 (=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

train_df = pd.read_csv('data/train.csv')
train_df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


## 데이터 전처리

In [2]:
def load_data() -> tuple:
    train_df = pd.read_csv('data/train.csv')
    test_df = pd.read_csv('data/test.csv')

    train_data = train_df.drop(['index', 'FLAG_MOBIL', 'credit'], axis=1)
    test_data = test_df.drop(['index', 'FLAG_MOBIL'], axis=1)

    train_data = prep_data(train_data)
    test_data = prep_data(test_data)

    return train_data, test_data

In [3]:
def prep_data(df) -> pd.DataFrame:

    data = df.copy()
    data = data[data['occyp_type'].notnull() | (data['DAYS_EMPLOYED'] > 0)]
    data['occyp_type'] = data['occyp_type'].fillna('Unemployeed')

    data['gender'].replace({'M':0,'F':1}, inplace=True)
    data['car'].replace({'N':0,'Y':1}, inplace=True)
    data['reality'].replace({'N':0,'Y':1}, inplace=True)

    data['child_num'] = data['child_num'].apply(lambda x: 10 if x > 10 else x)
    data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(lambda x: (x*-1)/365)
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(lambda x: (x*-1)/365)
    data['begin_month'] = data['begin_month'].apply(lambda x: (x*-1)/12)
    data['family_size'] = data['family_size'].apply(lambda x: 10 if x > 10 else x)

    name_dict = get_name_dict()
    data['income_type'].replace(name_dict['income'], inplace=True)
    data['edu_type'].replace(name_dict['edu'], inplace=True)
    data['family_type'].replace(name_dict['family'], inplace=True)
    data['house_type'].replace(name_dict['house'], inplace=True)
    data['occyp_type'].replace(name_dict['occyp'], inplace=True)

    return data

In [4]:
def get_name_dict() -> dict:
    name_dict = dict()

    name_dict['income'] = {'Working': 0, 'Commercial associate': 1, 'Pensioner': 2, 'State servant': 3, 'Student': 4}
    name_dict['edu'] = {'Secondary / secondary special': 0, 'Higher education': 1, 'Incomplete higher': 2,
                        'Lower secondary': 3, 'Academic degree': 4}
    name_dict['family'] = {'Married': 0, 'Single / not married': 1, 'Civil marriage': 2, 'Separated': 3, 'Widow': 4}
    name_dict['house'] = {'House / apartment': 0, 'With parents': 1, 'Municipal apartment': 2, 'Rented apartment': 3,
                            'Office apartment': 4, 'Co-op apartment': 5}
    name_dict['occyp'] = {'Unemployeed': 0, 'Laborers': 1, 'Core staff': 2, 'Sales staff': 3, 'Managers': 4, 'Drivers': 5,
                            'High skill tech staff': 6, 'Accountants': 7, 'Medicine staff': 8, 'Cooking staff': 9,
                            'Security staff': 10, 'Cleaning staff': 11, 'Private service staff': 12, 'Low-skill Laborers': 13,
                            'Waiters/barmen staff': 14, 'Secretaries': 15, 'Realty agents': 16, 'HR staff': 17, 'IT staff': 18}

    return name_dict

In [5]:
numerical_transformer = StandardScaler()
numerical_features = ['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']

categorical_transformer = OneHotEncoder(categories='auto')
categorical_features = ['gender', 'car', 'reality', 'child_num',
                        'income_type', 'edu_type', 'family_type', 'house_type',
                        'work_phone', 'phone', 'email', 'occyp_type', 'family_size']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])

pipe = Pipeline(steps=[('preprocessor', preprocessor)])

In [7]:
train_data, test_data = load_data()
train_data = pipe.fit_transform(train_data)
train_data.shape

(22724, 73)

In [14]:
train = pd.read_csv('data/test.csv')
train = train.drop(['index'], axis=1)
train = train[train['occyp_type'].notnull() | (train['DAYS_EMPLOYED'] > 0)]
train

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,M,Y,Y,0,131400.0,Pensioner,Secondary / secondary special,Married,House / apartment,-23646,365243,1,0,0,0,NaN,2.0,-3.0
9995,F,Y,Y,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,-5434,1,1,1,0,Accountants,2.0,-19.0
9996,M,Y,Y,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,-1315,1,1,0,0,Laborers,2.0,-34.0
9997,F,N,Y,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,-14018,1,0,0,0,Medicine staff,2.0,-55.0


In [11]:
test = pd.read_csv('data/test.csv')
test['child_num'].value_counts()

0    6861
1    2106
2     894
3     113
4      16
5      10
Name: child_num, dtype: int64

In [6]:
df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
1,1,1,0,1,1,247500.0,1,0,2,0,-11380,-1540,1,0,0,1,1,3.0,-5.0,1.0
2,2,0,1,1,0,450000.0,0,1,0,0,-19087,-4434,1,0,1,0,4,2.0,-22.0,2.0
3,3,1,0,1,0,202500.0,1,0,0,0,-15088,-2092,1,0,1,0,3,2.0,-37.0,0.0
4,4,1,1,1,0,157500.0,3,1,0,0,-15037,-2105,1,0,0,0,4,2.0,-26.0,2.0
5,5,1,0,1,2,270000.0,0,0,0,0,-13413,-4996,1,0,0,1,6,4.0,-18.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  int64  
 2   car            26457 non-null  int64  
 3   reality        26457 non-null  int64  
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  int64  
 7   edu_type       26457 non-null  int64  
 8   family_type    26457 non-null  int64  
 9   house_type     26457 non-null  int64  
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     26457 non-null  int64  
 17  family_size    26457 non-null  float64
 18  begin_

In [4]:
df.set_index('index').to_csv('data_prep.csv')